<div style="Tamplaged                                                                              ">
<b><center><font size="4">Notebook Default</font></center></b>
<b><center><font size="3">Title of the job</font></center></b>
<b><center><font size="2">one short statement about the job</font></center></b>


</div>

**Notebook Developed by**: [Rene Jerez](https://github.com/renejerez)<br>
**email:**  rene.jerez@libertyseguros.pt<br>

In [10]:
from general_tools import summary_create

In [17]:

# Create the summary writing the sections and subsections below:
sections = [
    ("Web Scraping and Data Extraction", [
        ("Open URLs, handle pop-ups", []),
        ("Extract page content, scores, and 'stars' elements", [])
    ]),
    ("Data Processing and Storage", [
        ("Parse and count star ratings", []),
        ("Store results in DataFrame", [
            ("Subsection 1", []),
            ("Subsection 2", [])
        ])
    ])
]

In [19]:
summary = summary_create(sections)
summary

'<h1>Scraping Scores and Number of Reviews<span class="tocSkip"></span></h1>\n<div class="toc">\n  <ul class="toc-item">\n    <li><span><a href="#Web-Scraping-and-Data-Extraction" data-toc-modified-id="Web-Scraping-and-Data-Extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Web Scraping and Data Extraction</a></span>\n      <div class="toc">\n        <ul class="toc-item">\n    <li><span><a href="#Open-URLs,-handle-pop-ups" data-toc-modified-id="Open-URLs,-handle-pop-ups-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Open URLs, handle pop-ups</a></span>\n    </li>\n    <li><span><a href="#Extract-page-content,-scores,-and-\'stars\'-elements" data-toc-modified-id="Extract-page-content,-scores,-and-\'stars\'-elements-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extract page content, scores, and \'stars\' elements</a></span>\n    </li>\n        </ul>\n      </div>\n    </li>\n    <li><span><a href="#Data-Processing-and-Storage" data-toc-modified-id="Data-Processin

<h1>Scraping Scores and Number of Reviews<span class="tocSkip"></span></h1>
<div class="toc">
  <ul class="toc-item">
    <li><span><a href="#Web-Scraping-and-Data-Extraction" data-toc-modified-id="Web-Scraping-and-Data-Extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Web Scraping and Data Extraction</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Open-URLs,-handle-pop-ups" data-toc-modified-id="Open-URLs,-handle-pop-ups-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Open URLs, handle pop-ups</a></span>
    </li>
    <li><span><a href="#Extract-page-content,-scores,-and-'stars'-elements" data-toc-modified-id="Extract-page-content,-scores,-and-'stars'-elements-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extract page content, scores, and 'stars' elements</a></span>
    </li>
        </ul>
      </div>
    </li>
    <li><span><a href="#Data-Processing-and-Storage" data-toc-modified-id="Data-Processing-and-Storage-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Processing and Storage</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Parse-and-count-star-ratings" data-toc-modified-id="Parse-and-count-star-ratings-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parse and count star ratings</a></span>
    </li>
    <li><span><a href="#Store-results-in-DataFrame" data-toc-modified-id="Store-results-in-DataFrame-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Store results in DataFrame</a></span>
      <div class="toc">
        <ul class="toc-item">
    <li><span><a href="#Subsection-1" data-toc-modified-id="Subsection-1-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Subsection 1</a></span>
    </li>
    <li><span><a href="#Subsection-2" data-toc-modified-id="Subsection-2-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Subsection 2</a></span>
    </li>
        </ul>
      </div>
    </li>
        </ul>
      </div>
    </li>
  </ul>
</div>


<hr>

# Scraping Scores and Number of Reviews

This script automates the process of scraping review scores and star ratings from a list of URLs stored in a CSV file. Using Selenium WebDriver, it navigates to each URL, handles potential pop-ups, and extracts relevant content using BeautifulSoup. The extracted data includes review scores and star ratings, which are then stored in a DataFrame for further analysis.

## Setup and Load Data

### Import libraries

In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re

### Load bodyshops from CSV file

In [15]:
# Load the CSV file
df = pd.read_csv('bodyshops_to_check.csv')

### Find URLs for each bodyshop

In [ ]:
# Step 1: Set up Selenium WebDriver
def setup_driver():
    options = Options()
    options.add_argument('--disable-gpu')  # Disable GPU acceleration
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Step 2: Open URL and handle pop-ups
def open_url_and_handle_popups(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    try:
        # Accept cookies pop-up
        accept_cookies_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Accept all']"))
        )
        accept_cookies_button.click()
        print("Accepted cookies")
    except Exception as e:
        print(f"No cookies button found: {e}")

    return driver

# Step 3: Perform search and get URL
def search_and_get_url(driver, address, name):
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "searchboxinput"))
    )
    search_box.clear()
    search_box.send_keys(address)
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)  # Wait for search results to load

    search_box.clear()
    search_box.send_keys(name)
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)  # Wait for search results to load

    current_url = driver.current_url
    return current_url

# Initialize the WebDriver
driver = setup_driver()
driver = open_url_and_handle_popups(driver, "https://www.google.com/maps")

# Iterate through the CSV and perform searches
urls = []
for index, row in df.iterrows():
    address = row['bodyshop_address']
    name = row['nome_prestador_oficina']
    url = search_and_get_url(driver, address, name)
    urls.append(url)

# Add the URLs to the DataFrame
df['url'] = urls

# Save the updated DataFrame back to the same CSV file
df.to_csv(csv_file_path, index=False)

# Close the WebDriver
driver.quit()

## WebDriver Configuration

### Configure and initialize Chrome WebDriver

In [16]:
# Step 1: Set up Selenium WebDriver
def setup_driver():
    options = Options()
    options.add_argument('--disable-gpu')  # Disable GPU acceleration
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Step 2: Open URL and handle pop-ups
def open_url_and_handle_popups(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    try:
        # Accept cookies pop-up
        accept_cookies_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Accept all']"))
        )
        accept_cookies_button.click()
        print("Accepted cookies")
    except Exception as e:
        print(f"No cookies button found: {e}")

    return driver

# Step 3: Extract page content and find the score
def extract_score(driver):
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Locate the score element using aria-label containing "stars"
    score_element = soup.find('span', {'aria-label': lambda x: x and 'stars' in x})
    score = "Score element not found"
    if score_element:
        score = score_element['aria-label']
        print(f"Score (Rating): {score}")
    return score

# Step 4: Extract elements containing 'stars'
def extract_star_elements(driver):
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Locate all elements containing 'stars' in aria-label
    star_elements_info = []
    star_elements = soup.find_all(attrs={'aria-label': lambda x: x and 'stars' in x})
    for element in star_elements:
        star_elements_info.append(f"Element tag: {element.name}, Classes: {element.get('class')}, Aria-label: {element['aria-label']}")
    return star_elements_info


New Columns

URL: Path of the Google Maps link for the body shop

Result: Raw HTML information extracted

Stars: Number of stars people rated the body shop

Score: The score people rated the body shop

Review: The number of people who provided a specific score for the body shop